In [10]:
import random
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from requests import Request, Session

In [11]:
class Portfolio:
    def __init__(self, cash, stocks={},):
        self.stocks = stocks
        self.cash = cash
        self.url = 'https://pro-api.coinmarketcap.com'
        self.api_key = '06b26ee1-737d-48ad-bd50-b9a770e1fc6e'
        self.headers = {
            'Accepts': 'application/json',
            'X-CMC_PRO_API_KEY': self.api_key,
        }
        self.session = Session()
        self.session.headers.update(self.headers)

    def total_value(self):
        total = 0
        for stock in self.stocks:
            total += stock.get_value()
        return total

    def buy_stock(self, stock):
        self.cash -= stock.get_value()
        self.stocks[stock.symbol] = stock
        print(f"Bought {stock.shares} of {stock.symbol} @ {stock.price}")
    
    def sell_random_stock(self):
        '''
        Sells a random stock that is currently owned by first getting the new value of all stocks then selling the stock to be sold. 
        '''
        self.update_stock_info()
        stock = random.choice(self.stocks)
        print(f"Sold {stock.shares} of {stock.symbol} @ {stock.price}")
        self.cash += stock.get_value()
        del self.stocks[stock.symbol]

    def update_stock_info(self):
        '''
        Updates the price of all stocks owned in the portfolio.        
        '''
        cryptos_to_update = ""
        for stock in self.stocks:
            cryptos_to_update += stock.symbol + ","
        response = None
        parameters = {
            'symbol': cryptos_to_update,
            'convert': 'USD'
        }
        try: 
            response = self.session.get(self.url + '/v1/cryptocurrency/quotes/latest', params=parameters)
            results = response.json()
            for stock in self.stocks:
                for result in results['data']:
                    if result['symbol'] == stock.symbol:
                        stock.price = result['quote']['USD']['price']
            print(self)
        except (ConnectionError, TimeoutError, TooManyRedirects) as e:
            print(e)
        
    
    def __str__(self):
        return str(f"{len(self.stocks)} coins owned with a total value of {self.total_value()} with ${self.cash} cash on hand.")

In [12]:
class Stock:
    def __init__(self, symbol, shares, price):
        self.symbol = symbol
        self.shares = shares
        self.price = price
    
    def get_value(self,):
        return self.shares * self.price
